<a href="https://colab.research.google.com/github/naitotomoyuki/2021-singakujutu/blob/master/yolo8learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [2]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/

ストリーミング出力は最後の 5000 行に切り捨てられました。
          1.80K 100%    0.00kB/s    0:00:00 (xfr#1489, to-chk=2499/5944)
labels/train/166233fa-Ch120241207233229.txt
            900 100%    1.35kB/s    0:00:00 (xfr#1490, to-chk=2498/5944)
labels/train/16a14d1e-Ch120241210094741.txt
            547 100%    0.40kB/s    0:00:01 (xfr#1491, to-chk=2497/5944)
labels/train/16cce3bd-Ch120241207094305.txt
            812 100%    0.00kB/s    0:00:00 (xfr#1492, to-chk=2496/5944)
labels/train/16e46f5a-Ch120241207120748.txt
          1.96K 100%    2.86kB/s    0:00:00 (xfr#1493, to-chk=2495/5944)
labels/train/16fa10c7-Ch120241207124102.txt
          1.72K 100%    1.30kB/s    0:00:01 (xfr#1494, to-chk=2494/5944)
labels/train/16fe91fc-Ch120241207124118.txt
          1.71K 100%    0.00kB/s    0:00:00 (xfr#1495, to-chk=2493/5944)
labels/train/172768a1-Ch120250107014033.txt
            986 100%    1.58kB/s    0:00:00 (xfr#1496, to-chk=2492/5944)
labels/train/1741b919-Ch120241207221956.txt
            899 100%    0.67kB/s

In [10]:
model_path = "/content/drive/MyDrive/yolov8s.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [11]:
import yaml


# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [ ]:
import os
from ultralytics import YOLO
import numpy as np


def train_yolov8(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLOv8 model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLOv8 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolov8_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=180,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
model_path = "/content/drive/MyDrive/yolov8s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 200  # エポック数
batch_size = 32  # バッチサイズ
img_size = 960  # 画像サイズ
iou_threshold = 0.3  # IoU 閾値
mosaic = 1.0  # モザイクの確率
mixup = 0.2  # Mixup の確率
output_dir = "runs_experiment"  # 出力ディレクトリ

# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLOv8 の学習実行
train_yolov8(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)


Loading model from /content/drive/MyDrive/yolov8s.pt
Training YOLOv8 with 150 epochs...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/yolov8s.pt, data=/content/dataset/data.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_experiment, name=yolov8_training3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.3, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

100%|██████████| 755k/755k [00:00<00:00, 75.4MB/s]


Overriding model.yaml nc=80 with nc=63

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 159MB/s]


AMP: checks passed ✅


...

    136/150      11.5G     0.9229     0.5011      1.121        450        960: 100%|██████████| 130/130 [00:27<00:00,  4.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.27it/s]


                   all        593      10600      0.945      0.926      0.942      0.679

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    137/150      9.68G     0.9238     0.4979      1.119        478        960: 100%|██████████| 130/130 [00:27<00:00,  4.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.39it/s]


                   all        593      10600      0.944      0.925      0.943      0.682

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    138/150      11.6G     0.9313     0.4991      1.113        523        960:  54%|█████▍    | 70/130 [00:14<00:11,  5.37it/s]